In [1]:
from helper_dataprocessing import *
import os
def create_directory(outdir,outname):
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    return os.path.join(outdir, outname)   
outdir = '/local/wujimeng/code_jimeng/data/invitro'

In [2]:
print("start loading the properties data...")
DATA_PROPERTY_PATH = [
    '/local/wujimeng/code_simone/data/DSSToxQueryWPred1.xlsx', '/local/wujimeng/code_simone/data/DSSToxQueryWPred2.xlsx',
    '/local/wujimeng/code_simone/data/DSSToxQueryWPred3.xlsx', '/local/wujimeng/code_simone/data/DSSToxQueryWPred4.xlsx'
]

df_property = pd.DataFrame()
for i in DATA_PROPERTY_PATH:
    df_property = pd.concat(
        [
            df_property,
            pd.read_excel(
                open(i, "rb"),
                usecols=[
                    "Substance_CASRN",
                    "Structure_SMILES",
                    "Structure_MolWt",
                    "NCCT_MP",
                    "NCCT_WS",
                ],
                engine="openpyxl",
            ),
        ],
        axis=0,
    )
    print(str(i) + " loaded", ctime())
df_property = df_property.rename(
    {
        "Substance_CASRN": "cas_number",
        "Structure_SMILES": "smiles",
        "Structure_MolWt": "mol_weight",
        "NCCT_MP": "melting_point",
        "NCCT_WS": "water_solubility",
    },
    axis=1,
)

start loading the properties data...
/local/wujimeng/code_simone/data/DSSToxQueryWPred1.xlsx loaded Fri Jul 23 11:55:20 2021
/local/wujimeng/code_simone/data/DSSToxQueryWPred2.xlsx loaded Fri Jul 23 11:57:43 2021
/local/wujimeng/code_simone/data/DSSToxQueryWPred3.xlsx loaded Fri Jul 23 11:59:59 2021
/local/wujimeng/code_simone/data/DSSToxQueryWPred4.xlsx loaded Fri Jul 23 12:01:49 2021


In [3]:
results = pd.read_csv("/local/wujimeng/jupyter_projects/data/raw/celltox_database_200902.csv").drop(columns = ['Unnamed: 0','corresponding_researcher','full_name','corresponding_researcher','experimenter','dosing_direct_or_indirect'])
# cas_property = pd.read_csv("data/invitro/cas_property_invitro.csv").drop(columns = ['Unnamed: 0'])
len(results.chemical_name.unique())

121

In [5]:
null_output_counts(results)
final_db = results.merge(df_property, on='cas_number')
len(final_db.chemical_name.unique())

103

In [7]:
from tqdm import tqdm
def smiles_to_pubchem(df):

    pubchem = pd.DataFrame()
    pubchem["smiles"] = df["smiles"].unique()
    pubchem["pubchem2d"] = np.nan
    for i in tqdm(range(pubchem.shape[0])):
        try:
            pubchem.loc[i, "pubchem2d"] = pcp.get_compounds(
                pubchem["smiles"][i], "smiles"
            )[0].cactvs_fingerprint
        except:
            pubchem.loc[i, "pubchem2d"] = np.nan
    df_pubchem = df.merge(pubchem, on="smiles")

    return df_pubchem

final_pub = smiles_to_pubchem(final_db)

100%|██████████| 102/102 [01:29<00:00,  1.14it/s]


In [10]:
final = final_pub[~final_pub.smiles.isnull()]
final = final[~final.pubchem2d.isnull()]
null_output_counts(final)
len(final.chemical_name.unique())

99

In [42]:
def extract_mol_properties(features):
    features = features[~features.smiles.isnull()]
    features = features[~features.pubchem2d.isnull()]
    chem_feat = adding_smiles_features(features)
    to_drop_nofeat = chem_feat[chem_feat["bonds_number"] == np.nan].index
    chem_feat.drop(to_drop_nofeat, inplace=True)
#     to_drop_null = chem_feat[chem_feat.isnull().any(axis=1)].index
#     chem_feat.drop(index=to_drop_null, inplace=True)
    return chem_feat

In [43]:
chemical = extract_mol_properties(final)
# chemical = chemical.drop(['melting_unit'],axis=1)

Finding atom number...
Finding number of alone atoms...
Finding single bounds number...
Finding double bounds number...
Finding triple bounds number...
Finding ring number...
Finding morgan density...
Finding partition number (LogP)...
Finding number of OH group...
Finding melting point...
Finding water solubility...


In [45]:
null_output_counts(chemical)
len(chemical.chemical_name.unique())

98

<font size=5>Remove null value

In [89]:
results_pre = chemical.copy()

In [90]:
for label in ['_log_kow','_solubility','_henry_constant']:
    results_pre = results_pre.drop(['experimental'+label,'estimated'+label,"user_corrected_experimental"+label],axis=1)

In [91]:
results_pre = results_pre.rename(columns = {"cas_number":"test_cas"})

In [92]:
results_pre.columns

Index(['exposure_id', 'chemical_name', 'test_cas', 'molecular_weight',
       'cell_line', 'endpoint', 'timepoint_hour', 'insert', 'passive_dosing',
       'plate_size', 'solvent', 'conc_determination_nominal_or_measured',
       'medium', 'fbs', 'ec50', 'ec50_ci_lower', 'ec50_ci_upper', 'ec10',
       'ec10_ci_lower', 'ec10_ci_upper', 'ntc', 'smiles', 'mol_weight',
       'melting_point', 'water_solubility', 'pubchem2d', 'atom_number',
       'alone_atom_number', 'bonds_number', 'doubleBond', 'tripleBond',
       'ring_number', 'MorganDensity', 'LogP', 'oh_count'],
      dtype='object')

In [97]:
results_pre = results_pre[~results_pre.water_solubility.isnull()]
results_pre = results_pre[~results_pre.molecular_weight.isnull()]
len(results_pre.chemical_name.unique())

92

In [98]:
results_pre['fbs'].fillna(0,inplace=True)
results_pre['ntc'].fillna(0,inplace=True)
len(results_pre.chemical_name.unique())

92

In [99]:
results_pre = results_pre[~results_pre.ec50.isnull()]
len(results_pre.chemical_name.unique())

92

In [101]:
# restictions on experiments conditions
results_pre =results_pre[results_pre.medium == 'L15/ex']
results_pre =results_pre[results_pre.timepoint_hour == 24]
results_pre =results_pre[(results_pre.cell_line == 'RTgill-W1')|(results_pre.cell_line =='RTgutGC')]

In [102]:
null_output_counts(results_pre)

,null_values_inc_NC_NR%,#outputs
exposure_id,0,658
chemical_name,0,80
test_cas,0,80
molecular_weight,0,77
cell_line,0,2
endpoint,0,4
timepoint_hour,0,1
insert,0,1
passive_dosing,0,1
plate_size,0,1


In [112]:
outname = 'invitro_processed.csv'
results_pre.to_csv(create_directory(outdir,outname))
results_ec50 = results_pre.drop(['ec10', 'ec10_ci_lower', 'ec10_ci_upper'],axis=1)
results_ec10 = results_pre.drop(['ec50', 'ec50_ci_lower', 'ec50_ci_upper'],axis=1)
outname = 'invitro_ec50.csv'
results_ec50.to_csv(create_directory(outdir,outname))
outname = 'invitro_ec10.csv'
results_ec10.to_csv(create_directory(outdir,outname))